# https://rawg.io/apidocs

In [1]:
! pip install python-dotenv

In [86]:
import os
from dotenv import load_dotenv
import requests
from requests.exceptions import HTTPError
import json
import pandas as pd

# you need to get api keys from the link above n order to access the api. 
#I can share mine in the group chat, But I don't want to upload to the github.

#load_dotenv()
#a=os.getenv("rawg")
#b=os.getenv("b")


## Get a list of all genre

## End point: list of game

In [87]:
def get_api_data(dates="",metacritic=""):
    url=f"https://api.rawg.io/api/games?key={a}" #put the api key here to replace {a}

    q={ "dates":dates,"page_size":40,"search_exact":True,"metacritic":metacritic}

    try:
        response = requests.request("GET", url, params=q)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'1HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'1Other error occurred: {err}')
    else:
        if int(response.json()["count"])//40==0:
            return int(response.json()["count"])//40
        else:
            return int(response.json()["count"])//40 + 1

In [88]:
def get_api_data1(dates="",metacritic=""):
    ret=[]
    page=get_api_data(dates, metacritic)
    url=f"https://api.rawg.io/api/games?key={a}"
    for i in range(1,page+1):
        

        q={ "dates":dates,"page_size":40,"search_exact":True,"metacritic":metacritic, "page":i}

        try:
            response = requests.request("GET", url, params=q)
            response.raise_for_status()
        except HTTPError as http_err:
            print(f'2HTTP error occurred: {http_err}')
        except Exception as err:
            print(f'2Other error occurred: {err}')
        else:
            for j in response.json()["results"]:
                ret.append(j)
    return ret

In [162]:
def game_df_creater(list_of_games=[]):
    import re
    df_list=[]
    for game in list_of_games:
        a={}
        a["name"]=game["name"]
        a["released_date"]=game["released"]
        if game["platforms"]==None:
            a["platform"]=" "
        else:
            platform_list=[]
            for k in game["platforms"]:
                platform_list.append(k["platform"]["name"])
            a["platform"]=platform_list
        a["overall_rating"]=game["rating"]
        if len(game["ratings"])>0:
            for rate in game["ratings"]:
                a[rate["title"]]=rate["count"]
        a["metacritic"]=game["metacritic"]
        a["suggestions_count"]=game["suggestions_count"]
        if game["tags"] is not None:
            tags=[]
            for tag in game["tags"]:
                if tag["name"].isalpha():
                    tags.append(tag["name"])
                    tags_final=["".join(re.findall(r"[a-zA-Z0-9]", i)).lower() for i in tags]
            tags_final=[i for i in tags_final if i]
            a["tags"]=tags_final
            
        df_list.append(a)
    return pd.DataFrame(df_list)

In [94]:
# you can change the dates and metacritic score here
# just give a range of date you want like yyyy-mm-dd, use "," to seperate the dates
# give a range from 0-100 for metacritic, use "," to seperate
df=get_api_data1(dates="2021-01-01,2021-12-31",metacritic="80,100")
len(df)
# this one gives you a list of game, as shown below

101

In [98]:
df

[{'slug': 'halo-infinite',
  'name': 'Halo Infinite',
  'playtime': 2,
  'platforms': [{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc'}},
   {'platform': {'id': 1, 'name': 'Xbox One', 'slug': 'xbox-one'}},
   {'platform': {'id': 186,
     'name': 'Xbox Series S/X',
     'slug': 'xbox-series-x'}}],
  'stores': [{'store': {'id': 1, 'name': 'Steam', 'slug': 'steam'}},
   {'store': {'id': 2, 'name': 'Xbox Store', 'slug': 'xbox-store'}}],
  'released': '2021-12-08',
  'tba': False,
  'background_image': 'https://media.rawg.io/media/games/e1f/e1ffbeb1bac25b19749ad285ca29e158.jpg',
  'rating': 3.86,
  'rating_top': 4,
  'ratings': [{'id': 4,
    'title': 'recommended',
    'count': 143,
    'percent': 45.54},
   {'id': 5, 'title': 'exceptional', 'count': 88, 'percent': 28.03},
   {'id': 3, 'title': 'meh', 'count': 59, 'percent': 18.79},
   {'id': 1, 'title': 'skip', 'count': 24, 'percent': 7.64}],
  'ratings_count': 306,
  'reviews_text_count': 6,
  'added': 4715,
  'added_by_status': {'yet

In [163]:
# put the variable you get from the precious function here into game_df_creater
b=game_df_creater(df)
b.head()

,name,released_date,platform,overall_rating,recommended,exceptional,meh,skip,metacritic,suggestions_count,tags
0,Halo Infinite,2021-12-08,"[PC, Xbox One, Xbox Series S/X]",3.86,143.0,88.0,59.0,24.0,84,868,"[singleplayer, multiplayer, atmospheric, coope..."
1,Forza Horizon 5,2021-11-08,"[PC, Xbox One, Xbox Series S/X]",4.43,94.0,150.0,18.0,6.0,92,527,"[singleplayer, multiplayer, atmospheric, coope..."
2,Resident Evil: Village,2021-05-07,"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",4.41,137.0,243.0,29.0,14.0,83,869,"[singleplayer, multiplayer, atmospheric, horro..."
3,It Takes Two,2021-03-26,"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",4.54,115.0,370.0,19.0,22.0,89,842,"[multiplayer, cooperative, exploration, magic,..."
4,Mass Effect: Legendary Edition,2021-05-14,"[PC, PlayStation 5, Xbox One, PlayStation 4, X...",4.63,37.0,153.0,11.0,4.0,88,694,"[singleplayer, atmospheric, rpg, classic, spac..."
